# Example of Parsing SNP touchstone files
This is meant to be an example of how to parse a touchstone file with n>2 ports -
__It now has been replaced by pyMeasure.Code.DataHandlers.TouchstoneModels.SNP(file_name)__

In [1]:
# First we must determine the number of ports, this can be passed or derived from the extension
import re
EXTENSION_PATTERN="s(?P<Number_Ports>\d+)p"
test_file_names=["test1.s1p","test2.s2p","test3.S3P","test4.s4p","test5.S5p"]
for name in test_file_names:
    match=re.search(EXTENSION_PATTERN,name.split(".")[-1],re.IGNORECASE)
    number_ports=match.groupdict()["Number_Ports"]
    print("The number of ports for {0} is {1}".format(name,number_ports))
def number_ports_from_file_name(file_name):
    "Returns the number of ports as an integer from a file_name."
    match=re.search(EXTENSION_PATTERN,file_name.split(".")[-1],re.IGNORECASE)
    number_ports=match.groupdict()["Number_Ports"]
    return int(number_ports)

The number of ports for test1.s1p is 1
The number of ports for test2.s2p is 2
The number of ports for test3.S3P is 3
The number of ports for test4.s4p is 4
The number of ports for test5.S5p is 5


In [2]:
from pyMeasure import *

The module smithplot was not found,please put it on the python path


C:\Anaconda2\lib\site-packages\matplotlib\__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [22]:
def combine_segments(segment_list):
    """Combines a list of lists that are segments (each segment is list of strings)
    and returns a single list of strings, segments are assumed to be the same length"""
    combined_list=[]
    for index,row in enumerate(segment_list[0]):
        new_row=""
        for segment in segment_list:
            new_row=new_row+segment[index]
        combined_list.append(new_row)
    return combined_list

def parse_combined_list(sparameter_string_list):
    """Parses a list of strings, where each element of the list is a string with a single sparameter
    Assumes the data delimiter is a single tab, and removes any white space at the begining and end of the string
    all values are returned as floats"""
    parsed_data=[]
    for row in sparameter_string_list:
        new_row=map(lambda x:float(x),re.split("[\s|,]+",row.rstrip().lstrip().replace("\n","\t")))
        parsed_data.append(new_row)
    return parsed_data

In [33]:
# next we read the file 
#file_path=r'C:\Share\MUF_TEST_FILES\MUF 35mm\Results\Solutions\Solution_0.s4p'
#file_path=r'C:\Share\MUF_TEST_FILES\MUF 35mm\Results\DUTs\DUT_Support\DUT_0.s2p'
file_path=r'C:\Anaconda2\Lib\site-packages\pyMeasure\Code\DataHandlers\Tests\B7_baseline_50ohm_OR2_10n0_4p0_REV2_EVB1_01new.s3p'
number_ports=number_ports_from_file_name(file_path)
# assign the number of lines per sparameter
if number_ports in [1,2]:
    number_lines_per_sparameter=1
elif number_ports in [3]:
    number_lines_per_sparameter=3
else:
    number_lines_per_sparameter=number_ports**2/4
    
in_file=open(file_path,'r')
lines=[]
data_lines=[]
option_line=""
for line in in_file:
    lines.append(line)
    #line=line.rstrip().lstrip()
    # if the line is just '\n' ignore it
    if line in ["","\n"]:
        pass
    #if the line is an option line collect it
    elif re.search(OPTION_LINE_PATTERN,line,re.IGNORECASE):
        option_line=line
        continue
    elif re.match(COMMENT_PATTERN,line,re.IGNORECASE):
        continue
    else:
        data_lines.append(line)
comments=collect_inline_comments(lines,begin_token="!",end_token="\n")
data_lines=strip_inline_comments(data_lines,begin_token="!",end_token="\n")
segments=[data_lines[i::number_lines_per_sparameter] for i in range(number_lines_per_sparameter)]
combined_list=combine_segments(segments)
data=parse_combined_list(combined_list)

In [15]:
COMMENT_PATTERN="![\s]*(?P<Comment>.+)\n"
test_string='! Ports=123'
if re.match(COMMENT_PATTERN,test_string):
    print("True")

In [19]:
segments[0][0]

'  10.0000 -0.0895 -0.81 -60.1224 120.03 -62.5912 -55.86\n'

In [34]:
print comments

[[' Channel 1', 0, 0], [' Ports=123', 1, 0], [' Points=2329', 2, 0], [' Topology=ALL_SE(123)', 3, 0]]


In [42]:
combined=combine_segments(segments)
data=parse_combined_list(combined)

In [28]:
data[0]

[1.0,
 -0.00222181036562542,
 -0.00862246938348929,
 0.0,
 0.0,
 0.952784072344549,
 0.131160019122158,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.0154649204599162,
 0.0114069015580531,
 0.0,
 0.0,
 -0.949270726273018,
 -0.183539049741435,
 0.952784072344549,
 0.131160019122158,
 0.0,
 0.0,
 0.00445253596800139,
 0.0410950645167683,
 0.0,
 0.0,
 0.0,
 0.0,
 0.169316839115153,
 0.896713978299591,
 0.0,
 0.0,
 0.00283792933086696,
 -0.00150220181779146]